# Preparing notebook and folders

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['PYTHONHASHSEED'] = '2'

In [2]:
import numpy as np
np.random.seed(18)
import matplotlib.pyplot as plt
from keras.models import load_model

import concurrent.futures
import functools

import time

from F4_func_file import *

/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.1 when it was built against 1.14.0, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [3]:
#Create folder for storing evaluated data files
if not os.path.exists("Evaluated_data"):
    os.mkdir("Evaluated_data")

## Load model and data

In [4]:
#Load the DAMN model
DAMN_model = load_model("DAMN_model.h5", custom_objects={"custom_mse_func": custom_mse_func, "custom_mae_func": custom_mae_func})

In [5]:
#Load the experimentally measured data
meas_data_in = np.load("Measured_data.npz")["LR_meas_data"]
meas_data_target = np.load("Measured_data.npz")["HR_meas_data"]
meas_shape = meas_data_in.shape
print("Measured data shape:", meas_shape)

#Load the simulated data wth parameters estimated from experiment
sim_data_in = np.load("Simulated_data/Simulated_data_low_res.npy")
sim_data_target = np.load("Simulated_data/Simulated_data_high_res.npy")
sim_shape = sim_data_in.shape
print("Simulated data shape:", sim_shape)

Measured data shape: (8, 1000, 50, 50)
Simulated data shape: (101, 100, 50, 50)


## Evaluate both methods

### DAMN model

#### Simulated data

In [6]:
#Adjust the data shapes for easier evaluation
sim_data_in_reshaped = np.reshape(sim_data_in, (sim_shape[0]*sim_shape[1], sim_shape[2], sim_shape[3], 1))
sim_data_target_reshaped = np.reshape(sim_data_target, (sim_shape[0]*sim_shape[1], sim_shape[2], sim_shape[3], 1))

print("Simulated data reshaped:", sim_data_in_reshaped.shape)

Simulated data reshaped: (10100, 50, 50, 1)


In [7]:
#Use the DAMN model to predict high-resolution images
DAMN_sim_output_reshaped = np.squeeze(DAMN_model.predict(sim_data_in_reshaped))

#And return to the original shape
DAMN_sim_output = DAMN_sim_output_reshaped.reshape((sim_shape[0], sim_shape[1], sim_shape[2], sim_shape[3]))

np.save("Evaluated_data/DAMN_Simulated_data_output.npy", DAMN_sim_output)

316/316 [==============================] - 20s 57ms/step


In [8]:
#Evaluate a chosen metric comparing the DAMN output with the corresponding target image
DAMN_sim_errors = Evaluate_metric(DAMN_sim_output, sim_data_target)

np.save("Evaluated_data/DAMN_Simulated_data_errors.npy", DAMN_sim_errors)

#### Measured data

In [9]:
#Adjust the data shapes for easier evaluation
meas_data_in_reshaped = np.reshape(meas_data_in, (meas_shape[0]*meas_shape[1], meas_shape[2], meas_shape[3], 1))
meas_data_target_reshaped = np.reshape(meas_data_target, (meas_shape[0]*meas_shape[1], meas_shape[2], meas_shape[3], 1))

print("Measured data reshaped:", meas_data_in_reshaped.shape)

Measured data reshaped: (8000, 50, 50, 1)


In [10]:
#Use the DAMN model to predict high-resolution images
DAMN_meas_output_reshaped = np.squeeze(DAMN_model.predict(meas_data_in_reshaped))

#And return to the original shape
DAMN_meas_output = DAMN_meas_output_reshaped.reshape((meas_shape[0], meas_shape[1], meas_shape[2], meas_shape[3]))

np.save("Evaluated_data/DAMN_Measured_data_output.npy", DAMN_meas_output)

250/250 [==============================] - 14s 58ms/step


In [11]:
#Evaluate a chosen metric comparing the DAMN output with the corresponding target image
DAMN_meas_errors = Evaluate_metric(DAMN_meas_output, meas_data_target)

np.save("Evaluated_data/DAMN_Measured_data_errors.npy", DAMN_meas_errors)

### RL algorithm

In [12]:
#To fasten the Richardson_lucy algorithm, we split the data evaluation among several CPU units using ProcessPoolExecutor from concurrent.futures
CPU_units_to_use = 20   #Set to 1 if you are not familiar with ProcessPoolExecutor or your CPU availability

#Specifying the PSF_width value separately estimated from calibration data; requirement for Richardson-Lucy algorithm
PSF_width_value = 2.05
kernel = Gauss_kernel(PSF_width_value)

#### Simulated data

In [13]:
#As the RL algorithm is very time consuming (matter of up to hours), you can choose to evaluate only a small portion of the data for the graph visualization
#Namely, 1/5 of data samples in every 4-nd point on the horizontal axis
use_all_sim_data = True             #Switch to False for evaluating only the reduced dataset 

if use_all_sim_data:
    RL_sim_data_in = sim_data_in
    RL_sim_data_target = sim_data_target
    RL_sim_output = np.zeros(sim_data_in.shape)
else:
    RL_sim_data_in = sim_data_in[::4,:20]
    RL_sim_data_target = sim_data_target[::4,:20]
    RL_sim_output = np.zeros(sim_data_in[::4,:20].shape)

print("Using the following Simulated data shape for RL algorithm:", RL_sim_data_in.shape)

Using the following Simulated data shape for RL algorithm: (101, 100, 50, 50)


##### The following cell runs the Richardson-Lucy algorithm, which might turn very time consuming

In [14]:
#The ProcessPoolExecutor calling RL_iteration_for_concurrent function to evaluate the RL_data_in_SNR data
start = time.time()
for i in range(RL_sim_data_in.shape[0]):
    start_i = time.time()
    with concurrent.futures.ProcessPoolExecutor(CPU_units_to_use) as pool:
        intermediate_func = functools.partial(RL_iteration_for_concurrent, kernel)
        res = pool.map(intermediate_func, RL_sim_data_in[i])
    RL_sim_output[i] = np.array(list(res))
    print("Finished iteration", i+1, "out of", RL_sim_data_in.shape[0], "in", np.round(time.time()-start_i, 2), "seconds (" + str(np.round(time.time()-start, 2)), "from start).")

np.save("Evaluated_data/RL_Simulated_data_output.npy", RL_sim_output)

Finished iteration 1 out of 101 in 13.98 seconds (13.98 from start).
Finished iteration 2 out of 101 in 15.75 seconds (29.73 from start).
Finished iteration 3 out of 101 in 14.12 seconds (43.86 from start).
Finished iteration 4 out of 101 in 14.21 seconds (58.07 from start).
Finished iteration 5 out of 101 in 14.23 seconds (72.3 from start).
Finished iteration 6 out of 101 in 14.63 seconds (86.93 from start).
Finished iteration 7 out of 101 in 13.88 seconds (100.81 from start).
Finished iteration 8 out of 101 in 15.61 seconds (116.42 from start).
Finished iteration 9 out of 101 in 14.33 seconds (130.75 from start).
Finished iteration 10 out of 101 in 15.64 seconds (146.39 from start).
Finished iteration 11 out of 101 in 15.58 seconds (161.97 from start).
Finished iteration 12 out of 101 in 18.25 seconds (180.22 from start).
Finished iteration 13 out of 101 in 16.05 seconds (196.27 from start).
Finished iteration 14 out of 101 in 16.05 seconds (212.33 from start).
Finished iteration 15 

In [15]:
#Evaluate a chosen metric comparing the DAMN output with the corresponding target image
RL_sim_errors = Evaluate_metric(RL_sim_output, RL_sim_data_target)

np.save("Evaluated_data/RL_Simulated_data_errors.npy", RL_sim_errors)

#### Measured data

In [16]:
#As the RL algorithm is very time consuming (matter of up to hours), you can choose to evaluate only a small portion of the data for the graph visualization
#Namely, 1/50 of data samples in every point on the horizontal axis
use_all_meas_data = True             #Switch to False for evaluating only the reduced dataset 

if use_all_meas_data:
    RL_meas_data_in = meas_data_in
    RL_meas_data_target = meas_data_target
    RL_meas_output = np.zeros(meas_data_in.shape)
else:
    RL_meas_data_in = meas_data_in[:,:20]
    RL_meas_data_target = meas_data_target[:,:20]
    RL_meas_output = np.zeros(meas_data_in[:,:20].shape)

print("Using the following Measured data shape for RL algorithm:", RL_meas_data_in.shape)

Using the following Measured data shape for RL algorithm: (8, 1000, 50, 50)


##### The following cell runs the Richardson-Lucy algorithm, which might turn very time consuming

In [17]:
#The ProcessPoolExecutor calling RL_iteration_for_concurrent function to evaluate the RL_data_in_SNR data
start = time.time()
for i in range(RL_meas_data_in.shape[0]):
    start_i = time.time()
    with concurrent.futures.ProcessPoolExecutor(CPU_units_to_use) as pool:
        intermediate_func = functools.partial(RL_iteration_for_concurrent, kernel)
        res = pool.map(intermediate_func, RL_meas_data_in[i])
    RL_meas_output[i] = np.array(list(res))
    print("Finished iteration", i+1, "out of", RL_meas_data_in.shape[0], "in", np.round(time.time()-start_i, 2), "seconds (" + str(np.round(time.time()-start, 2)), "from start).")

np.save("Evaluated_data/RL_Measured_data_output.npy", RL_meas_output)

Finished iteration 1 out of 8 in 123.4 seconds (123.4 from start).
Finished iteration 2 out of 8 in 152.07 seconds (275.47 from start).
Finished iteration 3 out of 8 in 206.23 seconds (481.7 from start).
Finished iteration 4 out of 8 in 295.54 seconds (777.24 from start).
Finished iteration 5 out of 8 in 451.87 seconds (1229.11 from start).
Finished iteration 6 out of 8 in 693.84 seconds (1922.95 from start).
Finished iteration 7 out of 8 in 1139.92 seconds (3062.86 from start).
Finished iteration 8 out of 8 in 1966.74 seconds (5029.6 from start).


In [18]:
#Evaluate a chosen metric comparing the DAMN output with the corresponding target image
RL_meas_errors = Evaluate_metric(RL_meas_output, RL_meas_data_target)

np.save("Evaluated_data/RL_Measured_data_errors.npy", RL_meas_errors)